In [1]:
from sympy import MatrixSymbol, Matrix, Identity

import sympy as sym
import pandas as pd

sym.init_printing()
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import control
import random

# IDENTIFICACIÓN DE SISTEMAS

Profesor: Jairo Alberto Cuéllar Guarnizo  
Programa: Ingeniería en Automatización y Control

# Introducción a la Optimización

## El problema de la optimización

La optimización se refiere a los parámetros desconocidos como variable de decisión ($\theta$), frente a una función objetivo que depende de los datos medidos y usualmente llamada Y. Por ejemplo para Mínimos cuadrados, el problema de optimización es minimizar el error existente entre el vector Y y el modelo de predicción $M(\theta)$. La función a minimizar sería:

$$f(\theta):=|Y - M(\theta)|^2$$

Para efectos de los ejercicios simplemente se llamará "x" a la variable de decisión desconocida en el problema de optimización.

$$\min_{x \epsilon D} f(x)$$

Donde D es un espacio vectorial $R^n$.

## Condiciones de optimización

### Condiciones de primer orden

Si $\bar{x}$ es un punto de minimización local de f es llamado "punto estacionario" si su gradiente es 0.

$$\bigtriangledown f(\bar{x}) = 0$$

El punto estacionario o la estacionariedad es necesaria pero no es suficiente condicion para la optimalidad. Por tanto es necesario que para la función de costo f(x) se evalúe las derivadas de orden superior o de segundo orden, con el fin de definir si el punto $\bar{x}$ es un minimizador o no.

### Condiciones de segundo orden

La Matriz Hessiana de f(x*) debe ser simétrica y positiva semidefinida.

$$\bigtriangledown^2 f(x^*) \geqslant 0$$

Esta inecuación matricial es idéntica a decir que todos los autovalores de la matriz Hessiana deben ser no negativos, pero podrían ser ceros. Por tal razón esta condición no es suficiente para evaluar si x* es un minimizador de f.

NOTA: La matriz Hessiana es:

![Matriz Hessiana](figuras\00_hessian_matrix.jpg)

### Estabilidad en presencia de perturbaciones

La Matriz Hessiana de f(x*) debe ser simétrica y positiva definida.

$$\bigtriangledown^2 f(x^*) > 0$$

Entonces x* es minizador local de f, adicionalmente es único y estable frente a pequeñas perturbaciones de f, por tanto existe una constante C tal que para pequeños $p \epsilon R^n$ se cumple:

$$|x^* - arg min(f(x) + p^T.x)|\leqslant C|p|$$

Esa última inecuación matricial, asegura la estabilidad bajo pequeñas perturbaciones (p) para puntos óptimos que satisfacen las condiciones de segundo orden. 

$$x^*(p):= arg min(f(x) + p^T.x)$$

Esta expresión anterior se puede reescribir de la siguiente manera buscando que sea mínima se deriva e iguala a 0:

$$x^*(p) = \bigtriangledown f(x) + p = 0 $$

Se debe tener en cuenta lo siguiente:

$$|x^*(0)-x^*(p)|< C|p|$$

Para valores pequeños de "p".

## Ejemplo para 2 variables
Para la siguiente función de 2 variables $x_1$ y $x_2$:
$$f(x_1, x_2) = x_1^2+10x_2^2$$
Y teniendo en cuenta un vector:
$$p = \begin{bmatrix} 
p_1\\p_2
\end{bmatrix}$$
Evalúe las condiciones de optimalidad de dicha función de costo.

* Paso 1: Calcular el gradiente de f(x)

$$\bigtriangledown f(x) =\begin{bmatrix} 
\frac{\partial f}{\partial x_1}\\\frac{\partial f}{\partial x_2}
\end{bmatrix} = \begin{bmatrix} 
2x_1\\20x_2
\end{bmatrix}$$

* Paso 2: Calcular la Matriz Hessiana de f(x)

$$\bigtriangledown^2 f(x) = \begin{bmatrix}
\frac{\partial ^2f}{\partial ^2x_1}&\frac{\partial ^2f}{\partial x_1\partial x_2}\\ 
\frac{\partial ^2f}{\partial x_2x_1}&\frac{\partial ^2f}{\partial ^2x_2} 
\end{bmatrix} = \begin{bmatrix}
2&0\\ 
0&20 
\end{bmatrix}$$

* Paso 3: Evaluando la estabilidad ante p

$$x^*(p) = \begin{bmatrix} 
2x_1\\20x_2
\end{bmatrix} + \begin{bmatrix} 
p_1\\p_2
\end{bmatrix} = \begin{bmatrix} 
0\\0
\end{bmatrix}$$

Se obtienen los valores de $x_1$ y $x_2$ evaluados en el vector p, así:

$$x^*(p) = \begin{bmatrix} 
x_1(p)\\x_2(p)
\end{bmatrix} = \begin{bmatrix} 
\frac{-p_1}{2}\\\frac{-p_2}{20}
\end{bmatrix}$$

* Conclusiones: El gradiente da un punto $x_1 = 0, x_2 = 0$, la matriz Hessiana es mayor que 0 y por tanto se concluye que hay estabilidad ante las perturbaciones.